In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import warnings

import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from conformal.dataframes import load_df, set_test_metrics_columns, op_without_index
from conformal.plot import set_notebook_options, plot_coverage, plot_metrics_on_real_world_datasets_with_separate_axes, savefig, plot_metrics_for_varying_trainingsize
from conformal.visu import map_model, map_dataset

set_notebook_options()
warnings.filterwarnings('ignore', 'All-NaN axis encountered')

exp_name = 'final'
path = Path('figures') / exp_name
results_path = Path('results_conf') / exp_name
torch.cuda.is_available()

## Loading

In [ ]:
df_raw = load_df(results_path)
set_test_metrics_columns(df_raw)

## Preprocessing

In [ ]:
def op(df):
    df = df.copy()
    df['model'] = df['model'].map(map_model)
    df['dataset'] = df['dataset'].map(map_dataset)
    return df

df = op_without_index(df_raw, op)

def relative_length(x):
    return x / x.min()

df['relative_length'] = df.groupby(list(set(df.index.names) - {'conformalizer', 'run_id'}), dropna=False)['specialized_length'].transform(relative_length)

## Metrics plots

In [ ]:
mpl.rcParams['axes.formatter.limits'] = (-4, 4)

for type, plot_df in df.groupby('type', dropna=False):
    plot_df = plot_df.query('cal_size.isna() or cal_size == 1024')
    for model, df_model in plot_df.groupby('model', dropna=False):
        plot_metrics_on_real_world_datasets_with_separate_axes(df_model.query('dataset != "Hawkes"'), alpha=0.2, type=type)
        savefig(path / 'real_world_metrics' / 'per_model' / f'{model}_{type}.pdf')

## Coverage per level plots

In [ ]:
for metric in ['coverage', 'wsc']:
    for type, df_type in df.groupby('type', dropna=False):
        df_type = df_type.query('cal_size.isna() or cal_size == 1024')
        for model, df_model in df_type.groupby('model', dropna=False):
            plot_coverage(df_model, metric, ncols=6, type=type)
            savefig(path / f'{metric}_per_level' / model / f'{type}.pdf')

## Others

In [ ]:
for type, plot_df in df.groupby('type', dropna=False):
    for model, df_model in plot_df.groupby('model', dropna=False):
        plot_metrics_for_varying_trainingsize(df_model, alpha=0.2, type=type)
        savefig(path / 'trainingsize' / f'train_{model}_{type}.pdf')